In [2]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
import random

import sys
sys.path.append("..")
from Network_delay.delayDistribution import randomDelay, randomDelayList, user_delay_normal_09

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/data'

In [2]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, "hex")
        if 0 < decoded_private_key < bitcoin.N:
            break
    # print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, "wif")
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + "01"
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, "hex"), "wif"
    )
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, "hex")
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = "02"
    else:
        compressed_prefix = "03"
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [28]:
num_user=5000

df_user = pd.DataFrame(columns=["user_id", "address"])
for i in range(num_user):
    df_user.loc[i] = [i, gen_addr()]

df_user["mean"]=randomDelayList(num_user)
df_user

,user_id,address,mean
0,0,1AqpKb6FWLbA3SSHXCahDEgyncDz9xEqXB,271.471995
1,1,1H3kdzkeBUB17LZDXUNco36JJeAkC4A8D,186.426692
2,2,1Hx79ZQpYfGw2JxJAhENrZ8pbPQdd3qMPn,176.521015
3,3,14SvZvup9FFYLUoN5Mqar4JMhWL1kQuphq,97.058073
4,4,1MzBGfaPMNFdP18uMSAi3cCPj8H2g69XNe,74.477938
...,...,...,...
4995,4995,1Gbge73aL7q9oh1VmNSKcDMMTC3ni6nYsT,240.916550
4996,4996,1DSpE49YWQ77scQ5ppFv97jmbPQz85jtLc,173.266087
4997,4997,1JdHrURqyQgKuwhzaQJ7P8dRiHSp87J1mh,89.988573
4998,4998,1Bpa5rBVNVvVGFSmQ8LnEWG8L21DvxJ6LK,213.984260


In [29]:
df_user.to_csv("user_distr.csv", index=False)

---

In [30]:
df_user=pd.read_csv("user_distr.csv")

df_user

,user_id,address,mean
0,0,1AqpKb6FWLbA3SSHXCahDEgyncDz9xEqXB,271.471995
1,1,1H3kdzkeBUB17LZDXUNco36JJeAkC4A8D,186.426692
2,2,1Hx79ZQpYfGw2JxJAhENrZ8pbPQdd3qMPn,176.521015
3,3,14SvZvup9FFYLUoN5Mqar4JMhWL1kQuphq,97.058073
4,4,1MzBGfaPMNFdP18uMSAi3cCPj8H2g69XNe,74.477938
...,...,...,...
4995,4995,1Gbge73aL7q9oh1VmNSKcDMMTC3ni6nYsT,240.916550
4996,4996,1DSpE49YWQ77scQ5ppFv97jmbPQz85jtLc,173.266087
4997,4997,1JdHrURqyQgKuwhzaQJ7P8dRiHSp87J1mh,89.988573
4998,4998,1Bpa5rBVNVvVGFSmQ8LnEWG8L21DvxJ6LK,213.984260


In [49]:
df_tx = pd.DataFrame(
        columns=[
            "send_timestamp",
            "recv_timestamp",
            "latency",
            "label",
            "from",
            "to",
            "value",
        ]
    )
    
n=1000
for _ in range(n):
    send_ts = datetime.datetime.now()
    
    from_user_idx=random.randint(0, len(df_user) - 1)
    mean=df_user.loc[from_user_idx, "mean"]
    
    latency=user_delay_normal_09(mean, range=mean/10)
    fluc = randomDelay()
    latency = 0.7 * latency + 0.3 * fluc
    
    recv_ts = send_ts + datetime.timedelta(seconds=latency)
    
    from_user = df_user.loc[from_user_idx, "address"]
    to_user = np.random.choice(df_user["address"].values)
    value = random.random() * 10
    label = 0
    
    df_tx.loc[len(df_tx)]=[send_ts, recv_ts, latency, label, from_user, to_user, value]
    
df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-11-23 11:59:10.884667,2022-11-23 11:59:11.042485,0.157818,0,1NiQrLmr6kQqAm4QifmFKK1oUnDe1cjRyR,1BBGTr2YT1Euj4zKqJgcpuEqg7Bi462Aa3,7.398834
1,2022-11-23 11:59:10.886841,2022-11-23 11:59:11.126001,0.239160,0,1K2pQLDRvnBzWPQSAZvFA9KxgRxKQuovaF,1JDhpsw5NtexueUhziKzoAzcUdJVh3Ymu,1.053846
2,2022-11-23 11:59:10.888316,2022-11-23 11:59:11.307195,0.418879,0,1Ar8EbNTRuo86egjmQPyh9JR76MKH2wzJ5,15H1huF54AB57hddkHG3SuD1upaJwWbWpE,8.622671
3,2022-11-23 11:59:10.889791,2022-11-23 11:59:11.413486,0.523695,0,1GNc157nyiYcA8NeDkqdZP1vcaopTLXr9f,13wXPnoxLq1wP3XhT9UhSAv89uwdoAM6VR,1.523304
4,2022-11-23 11:59:10.891563,2022-11-23 11:59:11.750984,0.859421,0,1PzpnPRpQJQWx5wyjLZMMYHder16GEPBMv,1AsGXmmUstxzNuCfZFKKHhE1QEVNABbjca,9.817320
...,...,...,...,...,...,...,...
995,2022-11-23 11:59:11.944790,2022-11-23 11:59:12.647345,0.702555,0,1ECxyeND4WePmWGvHe4AQXeMhU5CT47sz9,13XsNFFHx5bibW4xXk789GSGUU6AyM9iXJ,8.764572
996,2022-11-23 11:59:11.946257,2022-11-23 11:59:12.878785,0.932528,0,1BPKB3KSGpwPL5nNiaJDTdB9UJSfdAtyPj,1BLDDo6BUjXHRp3DmpPyyCYbcXSxtK4bQ4,9.421025
997,2022-11-23 11:59:11.947563,2022-11-23 11:59:12.603639,0.656076,0,1BPKB3KSGpwPL5nNiaJDTdB9UJSfdAtyPj,1GDmHWqMq9Y5mKbSqmXnrSHzMh4da6gPo9,6.469720
998,2022-11-23 11:59:11.948601,2022-11-23 11:59:12.671786,0.723185,0,1B8NpvynmKdMuardPyQS6LnWe47FytBk6s,1NZq3zA1bfeKhTP3huqRUuq2eC11qs1SA9,0.528645


In [4]:
import multiprocess as mp

def gen_time(time_list):
    time_list.append(datetime.datetime.now())

time_list = mp.Manager().list()
n = 100000
num_jobs_running = 500
for i in range(n // num_jobs_running):
    jobs = []
    for _ in range(num_jobs_running):
        proc = mp.Process(
            target=gen_time, args=(time_list,)
        )
        jobs.append(proc)

    for j in jobs:
        j.start()
    for j in jobs:
        j.join()

    print(f"Finished round {i+1}")

time_list=list(time_list)
print(time_list[:10])

Finished round 1
Finished round 2
Finished round 3
Finished round 4
Finished round 5
Finished round 6
Finished round 7
Finished round 8
Finished round 9
Finished round 10
Finished round 11
Finished round 12
Finished round 13
Finished round 14
Finished round 15
Finished round 16
Finished round 17
Finished round 18
Finished round 19
Finished round 20
Finished round 21
Finished round 22
Finished round 23
Finished round 24
Finished round 25
Finished round 26
Finished round 27
Finished round 28
Finished round 29
Finished round 30
Finished round 31
Finished round 32
Finished round 33
Finished round 34
Finished round 35
Finished round 36
Finished round 37
Finished round 38
Finished round 39
Finished round 40
Finished round 41
Finished round 42
Finished round 43
Finished round 44
Finished round 45
Finished round 46
Finished round 47
Finished round 48
Finished round 49
Finished round 50
Finished round 51
Finished round 52
Finished round 53
Finished round 54
Finished round 55
Finished round 56
F

In [5]:
df_user = pd.read_csv("./user_distr.csv")
user_list = df_user["address"].values
user_mean_list = df_user["mean"].values

latency_list=[]
recv_list=[]
label_list=[0]*n
from_user_list=[]
to_user_list=[]
value_list=[]
for i in range(n):
    from_user_idx = random.randint(0, len(user_list) - 1)
    mean = user_mean_list[from_user_idx]
    
    latency=user_delay_normal_09(mean, range=mean/10)
    fluc = randomDelay()
    latency = 0.7 * latency + 0.3 * fluc
    latency_list.append(latency)
    
    recv_ts=time_list[i] + datetime.timedelta(seconds=latency)
    recv_list.append(recv_ts)
    
    from_user = user_list[from_user_idx]
    to_user = random.choice(user_list)
    value = random.random() * 10
    
    from_user_list.append(from_user)
    to_user_list.append(to_user)
    value_list.append(value)
    
df_tx=pd.DataFrame()
df_tx["send_timestamp"]=time_list
df_tx["recv_timestamp"]=recv_list
df_tx["latency"]=latency_list
df_tx["label"]=label_list
df_tx["from"]=from_user_list
df_tx["to"]=to_user_list
df_tx["value"]=value_list

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-11-29 16:23:22.585033,2022-11-29 16:25:05.932015,103.346982,0,1PS3YtFKbSszNCM96b5R29Nsa25ucT3HV1,1KXqy1ay5jmbdBW2j9QMb9KtS3AHLgMnq5,8.459220
1,2022-11-29 16:23:22.587099,2022-11-29 16:24:55.522555,92.935456,0,1CoMgg7ibLM54bYJzmDkp4xa1PpHtwnfYW,14yBuLyVd6w5mZSMSgkBGSQkTSztGaC8k5,4.037339
2,2022-11-29 16:23:22.591148,2022-11-29 16:24:26.731863,64.140715,0,14hpyezpEXKmVaq33Z8UZPfL8k4LRKUVZv,1LUkDemdqhGua4Y5taBva8ThXF3TBFTbx6,6.047191
3,2022-11-29 16:23:22.594809,2022-11-29 16:24:34.921516,72.326707,0,1Pay8GijkkK3i85PbPVdLsoU2sRB7LsPxK,13wL3em4DxofBafcG7siw7VqvEGuPxMXnE,3.748740
4,2022-11-29 16:23:22.598451,2022-11-29 16:26:00.508195,157.909744,0,1GahRnkcGyLrKqqV3mL6m1UVCvvbxWUXme,1KRHKSUqN7yeruJm3NS4iqLxoDoYo5vrV7,3.681252
...,...,...,...,...,...,...,...
99995,2022-11-29 16:28:15.104668,2022-11-29 16:30:31.220991,136.116323,0,1NWtW554YhWCFCjrEM2h36BHEJrpq2aBSu,1BkRBE6iqy4z7u54UhVUBmieSuJyYPsDyN,3.024810
99996,2022-11-29 16:28:15.107419,2022-11-29 16:32:58.110041,283.002622,0,1GThUAZVjzijaFzKhUTpcvUpawEJorviV6,1467zErxQvPjg4wL7omE6k7Tw1MFjspMX8,6.226353
99997,2022-11-29 16:28:15.110441,2022-11-29 16:30:59.492184,164.381743,0,16HEZ6oAfkrKk67NodQeeeXAibuS4Sastd,1Fv7vAHAsb7PH76YQyQ2aLKyeaeHk2Jqxg,1.163211
99998,2022-11-29 16:28:15.113213,2022-11-29 16:29:58.010970,102.897757,0,1BezmypZ1kqA7uNsnwjkTzd4GK2M6cpk7s,1LB2W7Vo2KJp6rRnWixFh9PANu1iCkpU6X,3.853043


In [6]:
df_tx.sort_values(by="recv_timestamp", inplace=True)

last_send_ts=datetime.datetime(1, 1, 1, 1, 1, 1)
for index, row in df_tx.iterrows():
    if row["send_timestamp"]<last_send_ts:
        df_tx.loc[index, "label"]=1
    last_send_ts=row["send_timestamp"]
    
df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
565,2022-11-29 16:23:24.153151,2022-11-29 16:23:30.811812,6.658661,0,1Duq2yA5eUjXDio2Kw1NjXTNrPa4D8eYrN,1GZVx1yiXq6xsM32BvWQeDDFy2sUjtdXX8,7.842294
332,2022-11-29 16:23:23.496523,2022-11-29 16:23:34.249087,10.752564,1,1M28txHJgpB19WJWoPRYQ3HhnKq6duscRN,12xHuNgb9E29Q6C2XquyDAuMFzArm2Coar,8.827062
240,2022-11-29 16:23:23.228113,2022-11-29 16:23:35.321867,12.093754,1,1CzMcGEs4MWwnxVy2zdzSTMfDmmPJhrD9Q,16j3YzW1T4oig5XRwDHquWkEhNgkuJNJ3Y,3.513410
1917,2022-11-29 16:23:28.014706,2022-11-29 16:23:36.277962,8.263256,0,1FrUhG9VNq32P54DG1MfyQx77ZtoGEcbKb,13W6YGNzUBegcCGJrjswUjmGoNvv2T5V9q,1.053890
4338,2022-11-29 16:23:34.992305,2022-11-29 16:23:37.644144,2.651839,0,1NwkHnUqWp9vTzCrFGYWbTFxddrHh9pX1Q,1DRog6uCSLnDza9MBAW7idW4TU6mb4R9Jv,1.210408
...,...,...,...,...,...,...,...
99492,2022-11-29 16:28:13.684168,2022-11-29 16:33:16.796495,303.112327,0,1JDspafguBgfFh2o9MPwb9K5tcvbRY2wv6,1JN6WoqwU3GyWrWFMB9i8ncig2asif1NUq,8.101758
93513,2022-11-29 16:27:56.602793,2022-11-29 16:33:24.543967,327.941174,1,17SQRhQDXdsedMcUzP97TYGvAjKQsUkVSk,1FSg2EHXd8m4reMVCixw6Dj3XUumAAjWxo,5.116663
90974,2022-11-29 16:27:49.334386,2022-11-29 16:33:32.452420,343.118034,1,1NmgV8mWmexpjRxLnjdJfqVTRrxvAQC6PA,12A4zgfvhLxhYvSJjdhzKZujvRmLX4DSzp,7.947236
96843,2022-11-29 16:28:06.132589,2022-11-29 16:34:00.203750,354.071161,0,1NmgV8mWmexpjRxLnjdJfqVTRrxvAQC6PA,1HzvxqBFRfP1szHsdsXgSnxqYbTkSsw1Bm,2.114410


In [7]:
sum(df_tx["label"] == 1)

50024

In [8]:
df_tx.to_pickle(f"./tx_{len(df_tx)}_distr.pkl")

---

---

Generate training data

In [14]:
df_user=pd.read_csv("./user_distr.csv")

user_list=df_user["address"].values.tolist()

user_list[:5]

['1AqpKb6FWLbA3SSHXCahDEgyncDz9xEqXB',
 '1H3kdzkeBUB17LZDXUNco36JJeAkC4A8D',
 '1Hx79ZQpYfGw2JxJAhENrZ8pbPQdd3qMPn',
 '14SvZvup9FFYLUoN5Mqar4JMhWL1kQuphq',
 '1MzBGfaPMNFdP18uMSAi3cCPj8H2g69XNe']

In [15]:
def get_user_id(row):
    from_user_id=user_list.index(row["from"])
    to_user_id=user_list.index(row["to"])
    
    row["from_user_id"]=from_user_id
    row["to_user_id"]=to_user_id
    
    return row

In [16]:
df_data=df_tx.copy()
df_data=df_data.apply(get_user_id, axis=1)

df_data

,send_timestamp,recv_timestamp,latency,label,from,to,value,from_user_id,to_user_id
565,2022-11-29 16:23:24.153151,2022-11-29 16:23:30.811812,6.658661,0,1Duq2yA5eUjXDio2Kw1NjXTNrPa4D8eYrN,1GZVx1yiXq6xsM32BvWQeDDFy2sUjtdXX8,7.842294,2170,2370
332,2022-11-29 16:23:23.496523,2022-11-29 16:23:34.249087,10.752564,1,1M28txHJgpB19WJWoPRYQ3HhnKq6duscRN,12xHuNgb9E29Q6C2XquyDAuMFzArm2Coar,8.827062,819,879
240,2022-11-29 16:23:23.228113,2022-11-29 16:23:35.321867,12.093754,1,1CzMcGEs4MWwnxVy2zdzSTMfDmmPJhrD9Q,16j3YzW1T4oig5XRwDHquWkEhNgkuJNJ3Y,3.513410,2823,1126
1917,2022-11-29 16:23:28.014706,2022-11-29 16:23:36.277962,8.263256,0,1FrUhG9VNq32P54DG1MfyQx77ZtoGEcbKb,13W6YGNzUBegcCGJrjswUjmGoNvv2T5V9q,1.053890,3527,1686
4338,2022-11-29 16:23:34.992305,2022-11-29 16:23:37.644144,2.651839,0,1NwkHnUqWp9vTzCrFGYWbTFxddrHh9pX1Q,1DRog6uCSLnDza9MBAW7idW4TU6mb4R9Jv,1.210408,2940,4984
...,...,...,...,...,...,...,...,...,...
99492,2022-11-29 16:28:13.684168,2022-11-29 16:33:16.796495,303.112327,0,1JDspafguBgfFh2o9MPwb9K5tcvbRY2wv6,1JN6WoqwU3GyWrWFMB9i8ncig2asif1NUq,8.101758,4357,3912
93513,2022-11-29 16:27:56.602793,2022-11-29 16:33:24.543967,327.941174,1,17SQRhQDXdsedMcUzP97TYGvAjKQsUkVSk,1FSg2EHXd8m4reMVCixw6Dj3XUumAAjWxo,5.116663,3319,4431
90974,2022-11-29 16:27:49.334386,2022-11-29 16:33:32.452420,343.118034,1,1NmgV8mWmexpjRxLnjdJfqVTRrxvAQC6PA,12A4zgfvhLxhYvSJjdhzKZujvRmLX4DSzp,7.947236,1415,4169
96843,2022-11-29 16:28:06.132589,2022-11-29 16:34:00.203750,354.071161,0,1NmgV8mWmexpjRxLnjdJfqVTRrxvAQC6PA,1HzvxqBFRfP1szHsdsXgSnxqYbTkSsw1Bm,2.114410,1415,2109


In [17]:
df_data.drop(columns=["send_timestamp", "from", "to"], inplace=True)

df_data

,recv_timestamp,latency,label,value,from_user_id,to_user_id
565,2022-11-29 16:23:30.811812,6.658661,0,7.842294,2170,2370
332,2022-11-29 16:23:34.249087,10.752564,1,8.827062,819,879
240,2022-11-29 16:23:35.321867,12.093754,1,3.513410,2823,1126
1917,2022-11-29 16:23:36.277962,8.263256,0,1.053890,3527,1686
4338,2022-11-29 16:23:37.644144,2.651839,0,1.210408,2940,4984
...,...,...,...,...,...,...
99492,2022-11-29 16:33:16.796495,303.112327,0,8.101758,4357,3912
93513,2022-11-29 16:33:24.543967,327.941174,1,5.116663,3319,4431
90974,2022-11-29 16:33:32.452420,343.118034,1,7.947236,1415,4169
96843,2022-11-29 16:34:00.203750,354.071161,0,2.114410,1415,2109


In [18]:
df_data.to_pickle(f"./data_{len(df_data)}_distr.pkl")

In [19]:
df_data[["recv_timestamp", "from_user_id", "to_user_id", "value", "label"]].values[:5]

array([[Timestamp('2022-11-29 16:23:30.811812'), 2170, 2370,
        7.842293911475852, 0],
       [Timestamp('2022-11-29 16:23:34.249087'), 819, 879,
        8.827062236943453, 1],
       [Timestamp('2022-11-29 16:23:35.321867'), 2823, 1126,
        3.5134096545691387, 1],
       [Timestamp('2022-11-29 16:23:36.277962'), 3527, 1686,
        1.0538899984497274, 0],
       [Timestamp('2022-11-29 16:23:37.644144'), 2940, 4984,
        1.2104075015134486, 0]], dtype=object)